In [1]:
import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_squared_error

import numpy as np
import DataPreprocessing
import SmartPolynomials

Using TensorFlow backend.


In [2]:
combs = ['1,3','1,4','2,3','2,4','1,3','1,4','2,3','2,4','1,2,3','1,2,4','1,3,4','2,3,4']

In [46]:
#me_train, mom_train = DataPreprocessing.npy('LO/MG_uuuxux/me_4jet_1000000', 'LO/MG_uuuxux//mom_4jet_1000000', combs, frac=1)
#me_test, mom_test = DataPreprocessing.npy('LO/MG_uuuxux/me_4jet_100000', 'LO/MG_uuuxux/mom_4jet_100000', combs, frac=0.1)
me_train, mom_train = DataPreprocessing.npy('LO/MG_uuxgg/JOE_me_3000000.npy', 'LO/MG_uuxgg/JOE_mom_3000000.npy', combs, frac=1)
me_test, mom_test = DataPreprocessing.npy('LO/MG_uuxgg/JOE_me_500000.npy', 'LO/MG_uuxgg/JOE_mom_500000.npy', combs, frac=0.1)
me_train=me_train/10.0**50
me_test=me_test/10.0**50

In [7]:
print(me_train)

[1.87683058e+03 5.62481798e+02 7.87978876e+02 ... 8.87116146e-01
 2.21967918e+00 3.45411257e-02]


In [8]:
print(np.std(me_test))

460.014608567249


In [71]:
# Random Model from Tensorflow Website
def build_model(components=300):
  model = keras.Sequential([
    keras.layers.Dense(components, activation='elu', input_shape=[12,]), #could try swish??      
    keras.layers.Dense(components, activation='elu'),
    keras.layers.Dense(components, activation='elu'),
    keras.layers.Dense(components, activation='elu'),
      
    keras.layers.Dense(1, activation='linear') 
  ])

  sgd=tf.keras.optimizers.SGD(0.000000000005, momentum=0.995, nesterov=True)
  model.compile(loss='mse', 
                optimizer=sgd,
                metrics=['mse'])
  return model

In [72]:
step=1000000

model = build_model()

scaler = StandardScaler()

training_error = []
test_error = []
perc_error = []

for n in range(0,len(mom_train),step):
    poly_train = np.array([np.ndarray.flatten(np.array(element)) for element in mom_train[n:n+step,1:]])
    ##Rescale
    scaler.partial_fit(poly_train) #Rescale on training set
    poly_train = scaler.transform(poly_train)
    
    model.fit(poly_train,me_train[n:n+step], batch_size=64, epochs=100, shuffle=True)
        
    if(n%100000==0): 
        print(n)
    
        ##Testing Error
        lin_mse = 0
        lin_perc = 0
        
        for m in range(0,len(mom_test),step):
            print(m)
            
            poly_test = np.array([np.ndarray.flatten(np.array(element)) for element in mom_test[m:m+step,1:]])
            poly_test = scaler.transform(poly_test)

            me_predict = model.predict(poly_test, verbose=0) #Prediction on test set
            me_predict.shape = (len(me_predict))
            
            lin_mse += np.sum(mean_squared_error(me_test[m:m+step], me_predict))
            lin_perc += 100*np.sum(np.divide(np.abs(me_test[m:m+step] - me_predict), me_test[m:m+step]))
            print(np.divide(np.abs(me_test[m:m+step] - me_predict), me_test[m:m+step]).shape)
            
        test_error.append(np.sqrt(lin_mse*step/len(me_test)))
        
        u=model.evaluate(poly_train, me_train[n:n+step])[0]
        training_error.append(np.sqrt(u))
        perc_error.append(lin_perc/len(me_test))
              
        
        print(training_error)
        print(test_error)
        print(perc_error)

Epoch 1/100
1000000/1000000 [==============================]1000000/1000000 [==============================] - 53s 53us/step - loss: 8605376230.5510 - mean_squared_error: 8605376230.5510

Epoch 2/100
1000000/1000000 [==============================]1000000/1000000 [==============================] - 52s 52us/step - loss: 1474934624.1811 - mean_squared_error: 1474934624.1811

Epoch 3/100
1000000/1000000 [==============================]1000000/1000000 [==============================] - 53s 53us/step - loss: 357477762.5550 - mean_squared_error: 357477762.5550

Epoch 4/100
1000000/1000000 [==============================]1000000/1000000 [==============================] - 53s 53us/step - loss: 272971050.2932 - mean_squared_error: 272971050.2932

Epoch 5/100
1000000/1000000 [==============================]1000000/1000000 [==============================] - 52s 52us/step - loss: 202631632.4556 - mean_squared_error: 202631632.4556

Epoch 6/100
1000000/1000000 [==============================]100000

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1000000/1000000 [==============================]1000000/1000000 [==============================] - 52s 52us/step - loss: 62827392.2843 - mean_squared_error: 62827392.2843

Epoch 16/100
1000000/1000000 [==============================]1000000/1000000 [==============================] - 52s 52us/step - loss: 66778379.7331 - mean_squared_error: 66778379.7331

Epoch 17/100
1000000/1000000 [==============================]1000000/1000000 [==============================] - 52s 52us/step - loss: 58394475.3926 - mean_squared_error: 58394475.3926

Epoch 18/100
  47424/1000000 [>.............................]  47424/1000000 [>.............................] - ETA: 49s - loss: 40302898.2716 - mean_squared_error: 40302898.2716

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1000000/1000000 [==============================]1000000/1000000 [==============================] - 53s 53us/step - loss: 48047212.2449 - mean_squared_error: 48047212.2449

Epoch 22/100
1000000/1000000 [==============================]1000000/1000000 [==============================] - 53s 53us/step - loss: 43201495.4997 - mean_squared_error: 43201495.4997

Epoch 23/100
1000000/1000000 [==============================]1000000/1000000 [==============================] - 53s 53us/step - loss: 40284086.8214 - mean_squared_error: 40284086.8214

Epoch 24/100
 594368/1000000 [================>.............] 594368/1000000 [================>.............] - ETA: 21s - loss: 39341872.5244 - mean_squared_error: 39341872.5244

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1000000/1000000 [==============================]1000000/1000000 [==============================] - 53s 53us/step - loss: 3625298.9946 - mean_squared_error: 3625298.9946

Epoch 91/100
1000000/1000000 [==============================]1000000/1000000 [==============================] - 53s 53us/step - loss: 4144703.3104 - mean_squared_error: 4144703.3104

Epoch 92/100
1000000/1000000 [==============================]1000000/1000000 [==============================] - 53s 53us/step - loss: 3962311.9243 - mean_squared_error: 3962311.9243

Epoch 93/100
 384704/1000000 [==========>...................] 384704/1000000 [==========>...................] - ETA: 33s - loss: 3456985.2772 - mean_squared_error: 3456985.2772

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1000000/1000000 [==============================]1000000/1000000 [==============================] - 53s 53us/step - loss: 3653024.9142 - mean_squared_error: 3653024.9142

Epoch 98/100
1000000/1000000 [==============================]1000000/1000000 [==============================] - 53s 53us/step - loss: 3656681.0022 - mean_squared_error: 3656681.0022

Epoch 99/100
1000000/1000000 [==============================]1000000/1000000 [==============================] - 53s 53us/step - loss: 3403663.0403 - mean_squared_error: 3403663.0403

Epoch 100/100
 601408/1000000 [=================>............] 601408/1000000 [=================>............] - ETA: 21s - loss: 3449923.4807 - mean_squared_error: 3449923.4807

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [36]:
print(np.std(me_test))

460.014608567249


In [73]:
print(training_error)

[3079.3611583396646, 2627.1899755301765, 1657.3843221453646]


In [75]:
print(test_error)

[16164.552824766115, 14212.876163222896, 9659.26483517181]


In [76]:
print(perc_error)

[90.51359258831695, 90.11540765945274, 83.68865221600602]
